In [1]:
import numpy as np
import pandas as pd

%load_ext rpy2.ipython

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import os

def load_smooth_dev():
    """Load the smooth package in development mode"""
    smooth_path = "/home/filtheo/smooth/"
    
    ro.r(f'''
    if (!requireNamespace("devtools", quietly=TRUE)) {{
        install.packages("devtools", repos="https://cran.rstudio.com/")
    }}
    devtools::load_all("{smooth_path}")
    ''')
    
    print("Smooth package loaded in development mode")

load_smooth_dev()

ℹ Loading smooth
Exports from /home/filtheo/smooth/src/adamGeneral.cpp:
   RcppExport SEXP adamFitterWrap(arma::mat matrixVt, arma::mat& matrixWt, arma::mat& matrixF, arma::vec& vectorG, arma::uvec& lags, arma::umat& indexLookupTable, arma::mat& profilesRecent, char const& Etype, char const& Ttype, char const& Stype, unsigned int const& componentsNumberETS, unsigned int const& nSeasonal, unsigned int const& nArima, unsigned int const& nXreg, bool const& constant, arma::vec& vectorYt, arma::vec& vectorOt, bool const& backcast, unsigned int const& nIterations, bool const& refineHead, bool const& adamETS)
   RcppExport SEXP adamForecasterWrap(arma::mat& matrixWt, arma::mat& matrixF, arma::uvec& lags, arma::umat& indexLookupTable, arma::mat& profilesRecent, char const& E, char const& T, char const& S, unsigned int const& componentsNumberETS, unsigned int const& nSeasonal, unsigned int const& nArima, unsigned int const& nXreg, bool const& constant, unsigned int const& horizon)
   RcppExport

R[write to console]: Loading required package: greybox

R[write to console]: Package "greybox", v2.0.3 loaded.


R[write to console]: This is package "smooth", v4.3.1.41008


R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/lib/R/site-library’ contains no packages



Smooth package loaded in development mode


### Test 1: Global level ETS(A,N,N) - backcasting vs optimal

In [2]:
np.random.seed(33)
n_points = 100
time_series = np.random.normal(100, 10, n_points)
ts_df = pd.DataFrame({'value': time_series})

In [3]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'optimal')
cat('ETS(A,N,N) with optimal initial:\n')
forecast(model, h = 12) |> print()

ETS(A,N,N) with optimal initial:
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61957 99.61957 99.61957 99.61957 99.61957 99.61957 99.61957 99.61957
 [9] 99.61957 99.61957 99.61957 99.61957


In [4]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'backcasting', nIterations = 1)
cat('ETS(A,N,N) with backcasting (nIterations=1):\n')
forecast(model, h = 12) |> print()

ETS(A,N,N) with backcasting (nIterations=1):
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 101.7634 101.7634 101.7634 101.7634 101.7634 101.7634 101.7634 101.7634
 [9] 101.7634 101.7634 101.7634 101.7634


In [5]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'backcasting', nIterations = 2)
cat('ETS(A,N,N) with backcasting (nIterations=2):\n')
forecast(model, h = 12) |> print()

ETS(A,N,N) with backcasting (nIterations=2):
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 101.1308 101.1308 101.1308 101.1308 101.1308 101.1308 101.1308 101.1308
 [9] 101.1308 101.1308 101.1308 101.1308


### Test 2: Local level ETS(A,N,N)

In [6]:
np.random.seed(41)
n_points = 120
errors = np.random.normal(0, 10, n_points)
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series})

In [7]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'optimal')
cat('ETS(A,N,N) with optimal initial:\n')
forecast(model, h = 12) |> print()

ETS(A,N,N) with optimal initial:
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 84.70888 84.70888 84.70888 84.70888 84.70888 84.70888 84.70888 84.70888
 [9] 84.70888 84.70888 84.70888 84.70888


In [8]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'backcasting', nIterations = 2)
cat('ETS(A,N,N) with backcasting (nIterations=2):\n')
forecast(model, h = 12) |> print()

ETS(A,N,N) with backcasting (nIterations=2):
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 84.71062 84.71062 84.71062 84.71062 84.71062 84.71062 84.71062 84.71062
 [9] 84.71062 84.71062 84.71062 84.71062


### Test 3: Local trend ETS(A,A,N)

In [9]:
np.random.seed(42)
n_points = 120
errors = np.random.normal(0, 10, n_points)
trend = np.random.normal(0.5, 2, n_points)
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + trend[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series})

In [10]:
%%R -i ts_df

model <- adam(ts_df, model = "AAN", lags = c(12), persistence = c(0.1, 0.02), initial = 'optimal')
cat('ETS(A,A,N) with optimal initial:\n')
forecast(model, h = 12) |> print()

ETS(A,A,N) with optimal initial:
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.2555 165.7640 166.2724 166.7808 167.2892 167.7977 168.3061 168.8145
 [9] 169.3230 169.8314 170.3398 170.8483


In [11]:
%%R -i ts_df

model <- adam(ts_df, model = "AAN", lags = c(12), persistence = c(0.1, 0.02), initial = 'backcasting', nIterations = 2)
cat('ETS(A,A,N) with backcasting (nIterations=2):\n')
forecast(model, h = 12) |> print()

ETS(A,A,N) with backcasting (nIterations=2):
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.2564 165.7647 166.2730 166.7813 167.2897 167.7980 168.3063 168.8146
 [9] 169.3229 169.8313 170.3396 170.8479


### Test 4: Damped trend ETS(A,Ad,N)

In [12]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdN", lags = c(12), initial = 'optimal')
cat('ETS(A,Ad,N) with optimal initial:\n')
print(model$B)
forecast(model, h = 12) |> print()

ETS(A,Ad,N) with optimal initial:
       alpha         beta          phi        level        trend 
  0.16827523   0.01042486   0.98630131 104.77485790  -0.38092112 
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 164.8748 165.4807 166.0783 166.6676 167.2489 167.8223 168.3878 168.9455
 [9] 169.4956 170.0382 170.5733 171.1011


In [13]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdN", lags = c(12), initial = 'backcasting', nIterations = 2)
cat('ETS(A,Ad,N) with backcasting (nIterations=2):\n')
print(model$B)
forecast(model, h = 12) |> print()

ETS(A,Ad,N) with backcasting (nIterations=2):
      alpha        beta         phi 
0.173071587 0.009699241 0.990062357 
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.0703 165.7054 166.3342 166.9567 167.5730 168.1832 168.7873 169.3854
 [9] 169.9776 170.5639 171.1444 171.7191


### Test 5: Multiplicative error ETS(M,N,N)

In [14]:
%%R -i ts_df

model <- adam(ts_df, model = "MNN", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(M,N,N) with optimal initial:\n')
forecast(model, h = 12) |> print()

ETS(M,N,N) with optimal initial:
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 163.0357 163.0357 163.0357 163.0357 163.0357 163.0357 163.0357 163.0357
 [9] 163.0357 163.0357 163.0357 163.0357


In [15]:
%%R -i ts_df

model <- adam(ts_df, model = "MNN", lags = c(12), distribution = 'dnorm', initial = 'backcasting', nIterations = 2)
cat('ETS(M,N,N) with backcasting (nIterations=2):\n')
forecast(model, h = 12) |> print()

ETS(M,N,N) with backcasting (nIterations=2):
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 163.0336 163.0336 163.0336 163.0336 163.0336 163.0336 163.0336 163.0336
 [9] 163.0336 163.0336 163.0336 163.0336


### Test 6: Multiplicative with additive trend ETS(M,A,N)

In [16]:
%%R -i ts_df

model <- adam(ts_df, model = "MAN", lags = c(12), persistence = c(0.1, 0.02), distribution = 'dnorm', initial = 'optimal')
cat('ETS(M,A,N) with optimal initial:\n')
forecast(model, h = 12) |> print()

ETS(M,A,N) with optimal initial:
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.2563 165.7645 166.2728 166.7811 167.2894 167.7977 168.3060 168.8143
 [9] 169.3226 169.8309 170.3392 170.8475


In [17]:
%%R -i ts_df

model <- adam(ts_df, model = "MAN", lags = c(12), persistence = c(0.1, 0.02), distribution = 'dnorm', initial = 'backcasting', nIterations = 2)
cat('ETS(M,A,N) with backcasting (nIterations=2):\n')
forecast(model, h = 12) |> print()

ETS(M,A,N) with backcasting (nIterations=2):
Time Series:
Start = 

121 
End = 132 
Frequency = 1 
 [1] 165.2564 165.7647 166.2730 166.7813 167.2897 167.7980 168.3063 168.8146
 [9] 169.3229 169.8313 170.3396 170.8479


### Test 7: Multiplicative damped trend ETS(M,Ad,N)

In [18]:
%%R -i ts_df

model <- adam(ts_df, model = "MAdN", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(M,Ad,N) with optimal initial:\n')
print(model$B)
forecast(model, h = 12) |> print()

ETS(M,Ad,N) with optimal initial:
       alpha         beta          phi        level        trend 
  0.14969093   0.01327518   0.96754121 105.75620898  -0.46234686 
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 164.1432 164.6555 165.1511 165.6307 166.0946 166.5436 166.9779 167.3982
 [9] 167.8048 168.1982 168.5789 168.9472


In [19]:
%%R -i ts_df

model <- adam(ts_df, model = "MAdN", lags = c(12), distribution = 'dnorm', initial = 'backcasting', nIterations = 2)
cat('ETS(M,Ad,N) with backcasting (nIterations=2):\n')
print(model$B)
forecast(model, h = 12) |> print()

ETS(M,Ad,N) with backcasting (nIterations=2):
     alpha       beta        phi 
0.15600482 0.01266795 0.97163619 
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 164.2911 164.8175 165.3289 165.8259 166.3087 166.7779 167.2338 167.6767
 [9] 168.1071 168.5252 168.9315 169.3263


### Test 8: Seasonal data ETS(A,A,A)

In [20]:
np.random.seed(42)
n_points = 120
errors = (1+np.random.normal(0, 0.1, n_points))
trend = np.random.normal(0.5, 2, n_points)
seasonal_sd = 0.2
seasonal_pattern = np.exp(np.random.normal(0, seasonal_sd, 12))
seasonal_pattern = seasonal_pattern / np.mean(seasonal_pattern)
time_series = np.zeros(n_points)
time_series[0] = 200 * seasonal_pattern[0] * errors[0]
for i in range(n_points-1):
    time_series[i+1] = ((time_series[i] / seasonal_pattern[(i) % 12]-trend[i]) * errors[i] ** (0.1-1) + trend[i+1]) * seasonal_pattern[(i+1) % 12] * errors[i+1]
ts_df = pd.DataFrame({'value': time_series})

In [21]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(A,A,A) with optimal initial:\n')
print(model$B)
forecast(model, h = 12) |> print()

ETS(A,A,A) with optimal initial:
        alpha          beta         gamma         level         trend 
 2.161811e-02  1.579075e-02  5.859355e-04  2.218595e+02 -7.471062e-01 
   seasonal_1    seasonal_2    seasonal_3    seasonal_4    seasonal_5 
-2.789793e+01 -1.110634e+01 -1.877190e-01  3.397825e+01 -4.309394e+01 
   seasonal_6    seasonal_7    seasonal_8    seasonal_9   seasonal_10 
 1.168707e+00 -1.501950e+01 -2.036129e+01  7.238855e+01  9.665323e+00 
  seasonal_11 
-3.498305e+01 
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 139.4253 157.0797 168.9017 203.9372 127.7433 172.8483 157.5823 153.1201
 [9] 246.7386 184.9105 141.1466 212.4595


In [22]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'backcasting', nIterations = 2)
cat('ETS(A,A,A) with backcasting (nIterations=2):\n')
print(model$B)
forecast(model, h = 12) |> print()

ETS(A,A,A) with backcasting (nIterations=2):
     alpha 

      beta      gamma 
0.04116081 0.01328189 0.00000000 
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 140.2917 154.9612 166.7535 199.7831 126.8559 165.6047 155.1955 149.6675
 [9] 242.2069 184.2166 138.1580 208.3858


### Test 9: Seasonal damped ETS(A,Ad,A)

In [23]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdA", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(A,Ad,A) with optimal initial:\n')
forecast(model, h = 12) |> print()

ETS(A,Ad,A) with optimal initial:
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 143.3686 159.1775 170.6195 206.3118 128.7002 165.9054 158.7444 152.9272
 [9] 245.0987 184.7617 139.8602 212.0961


In [24]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdA", lags = c(12), distribution = 'dnorm', initial = 'backcasting', nIterations = 2)
cat('ETS(A,Ad,A) with backcasting (nIterations=2):\n')
forecast(model, h = 12) |> print()

ETS(A,Ad,A) with backcasting (nIterations=2):
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 140.9647 155.5122 167.1765 200.0722 127.0053 165.6086 155.0481 149.3633
 [9] 241.7402 183.5821 137.3502 207.3993


### Test 10: Multiplicative seasonal ETS(M,A,M)

In [25]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(M,A,M) with optimal initial:\n')
print(model$B)
forecast(model, h = 12) |> print()

ETS(M,A,M) with optimal initial:
       alpha         beta        gamma        level        trend   seasonal_1 
1.481591e-01 1.426804e-02 7.384620e-04 2.037820e+02 6.272065e-01 8.650025e-01 
  seasonal_2   seasonal_3   seasonal_4   seasonal_5   seasonal_6   seasonal_7 
9.461651e-01 1.032165e+00 1.201518e+00 7.706866e-01 9.885534e-01 9.266625e-01 
  seasonal_8   seasonal_9  seasonal_10  seasonal_11 
9.073598e-01 1.417307e+00 1.067171e+00 8.234215e-01 
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 146.5487 160.5128 175.3330 204.3908 131.2754 168.6074 158.2698 155.1663
 [9] 242.7110 183.0018 141.3871 212.3149


In [26]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'backcasting', nIterations = 2)
cat('ETS(M,A,M) with backcasting (nIterations=2):\n')
print(model$B)
forecast(model, h = 12) |> print()

ETS(M,A,M) with backcasting (nIterations=2):
       alpha         beta        gamma 
3.720372e-02 1.232019e-02 2.261628e-07 
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 142.0638 155.2112 166.0928 197.8566 128.3186 165.0937 155.3297 149.0600
 [9] 237.4082 182.6504 138.4841 205.5224


### Test 11: Multiplicative seasonal damped ETS(M,Ad,M)

In [27]:
%%R -i ts_df

model <- adam(ts_df, model = "MAdM", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(M,Ad,M) with optimal initial:\n')
forecast(model, h = 12) |> print()

ETS(M,Ad,M) with optimal initial:
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 145.2918 161.0047 173.9070 202.6789 129.8380 167.2848 157.8726 154.0776
 [9] 241.7977 181.0301 139.8114 210.1557


In [28]:
%%R -i ts_df

model <- adam(ts_df, model = "MAdM", lags = c(12), distribution = 'dnorm', initial = 'backcasting', nIterations = 2)
cat('ETS(M,Ad,M) with backcasting (nIterations=2):\n')
forecast(model, h = 12) |> print()

ETS(M,Ad,M) with backcasting (nIterations=2):
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 143.2382 156.2763 166.9970 198.6501 128.6473 165.2754 155.2716 148.7827
 [9] 236.6104 181.7605 137.5986 203.8929
